ISYE 6999 - Deterministic Optimization

Homework 12 - Graham Billey

### Dantzig-Wolfe Decomposition

Consider the following lienar programming problem:

![](1.png)

**1) First, argue that the polyhedron $P$ defined above is bounded.** Hint: Can any variable $x_{ij}$ take infinte values without violating some constraints in $P$?

The set of constraints defined by $x_{ij} \geq 0 \,\, \forall i=1,2 \,\, j=1,2,3$ means that each variable is bounded from below by 0.

To see that each constraint is bounded from above, we can look at the remaining easy constraints.

$x_{11} + x_{12} + x_{13} = 20$ means that $x_{11}$, $x_{12}$, and $x_{13}$ can be at most 20. This occurs when exactly one of the three variables is nonzero.

$x_{21} + x_{22} + x_{23} = 10$ means that $x_{21}$, $x_{22}$, and $x_{23}$ can be at most 10. This occurs when exactly one of the three variables is nonzero.

**Since each variable is bounded from above and below, the polyhedron $P$ is bounded.**

----------------
**2) Since $P$ is bounded, we can use the extreme point representation for $P$.** The Dantzig-Wolfe master problem can be written as:

![](2.png)

**Specify $c$, $D$, and $b$ for this problem**

$c = (0, 2, 0, 0, 1, 1)^T$

$D = (1, 0, 0, 0, 0, 1)$

$b = 8$


-------------
**3) You are given the following two extreme points of the ployhedron $P$:**

$$ x^1 = (10, 0, 10, 0, 10, 0)^T $$

$$ x^2 = (0, 10, 10, 10, 0, 0)^T $$

**Construct the RMP using these two extreme points.** Use variables $\lambda_1$ and $\lambda_2$ for the RMP.



The RMP becomes:

$$ \max \lambda_1 c^T x^1 + \lambda_2 c^T x^2 $$

$$ s.t. \lambda_1 D x^1 + \lambda_2 D x^2 \leq 8 $$

$$ \lambda_1 + \lambda_2 = 1 $$

$$ \lambda_1, \lambda_2 \geq 0 $$

which simplifies to...

$$ \max 10 \lambda_1 + 20 \lambda_2 $$

$$ s.t. 10 \lambda_1 \leq 8 $$

$$ \lambda_1 + \lambda_2 = 1 $$

$$ \lambda_1, \lambda_2 \geq 0 $$

**4) Find the optimal solution to the RMP, which can be solved by hand.**

The optimal solution to the above RMP is $(\lambda_1, \lambda_2) = (0, 1)$.

The optimal objective value is $20$.

**5) Find the optimal dual variables. In order to do this, form the dual problem of the RMP and compute the optimal dual variables using primal and dual complimentary slackness.**

The RMP has the basis matrix $B = \begin{bmatrix} 10 & 0 \\ 1 & 1 \end{bmatrix}$, and cost vector $c_B = [10, 20]$.

The dual variables $[\hat y^T, \hat r] = c_B^T B^{-1}$. In the cell below we will calculate the dual variables.

The dual variables $[\hat y^T, \hat r] = [-1, 20]$.

In [1]:
import numpy as np

In [4]:
c_B = np.array([10, 20])
B = np.array([[10, 0],
              [1,  1]])

dual_vars = c_B.T @ np.linalg.inv(B)
print(f'Dual Variables: {dual_vars}')

Dual Variables: [-1. 20.]


**6) Using the dual variables computed in the previous part, formulate the subproblem that maximizes the reduced cost of the RMP.**

The pricing problem can be formulated as:

![](3.png)

In this case, $c^T - \hat y^T D = (1, 2, 0, 0, 1, 2)$, 

$\hat r = 20$, 

$F = \begin{bmatrix} 1 & 1 & 1 & 0 & 0 & 0 \\ 
                     0 & 0 & 0 & 1 & 1 & 1 \\
                     1 & 0 & 0 & 1 & 0 & 0 \\
                     0 & 1 & 0 & 0 & 1 & 0 \\
                     0 & 0 & 1 & 0 & 0 & 1 \\
                     \end{bmatrix}$, and

$\vec b = (20, 10, 10, 10, 10)^T$


**7) Solve the above subproblem using Python. Find the optimal solution and the optimal objective value. Be careful that the objective function may include a constant term. Should we terminate the Dantzig-Wolfe decomposition algorithm in this iteration?**

In [5]:
import cvxpy as cp

In [9]:
x = cp.Variable(6)

c = np.array([1,2,0,0,1,2])

F = np.array([[1,1,1,0,0,0],
              [0,0,0,1,1,1],
              [1,0,0,1,0,0],
              [0,1,0,0,1,0],
              [0,0,1,0,0,1]])

b = np.array([20,10,10,10,10])

obj = cp.Maximize(c@x - 20)
prob = cp.Problem(obj, [F @ x == b,
                        x >= 0])
prob.solve()

print(f'Objective function value at optimum: {prob.value}')
print(f'Optimal solution: {np.round(x.value,1)}^T')

if prob.value > 0:
    print('\nAdding this solution to the RMP will increase the objective value, so we continue iterating.')
else:
    print('\nAdding this solution to the RMP will not increase the objective value, \
    so we terminate the algorithm at this iteration!')

Objective function value at optimum: 30.0
Optimal solution: [10. 10.  0. -0. -0. 10.]^T

Adding this solution to the RMP will increase the objective value, so we continue iterating.


**8) Use the optimal solution from the above subproblem to form a new restricted master problem.**

$$ x^1 = (10, 0, 10, 0, 10, 0)^T $$

$$ x^2 = (0, 10, 10, 10, 0, 0)^T $$

$$ x^3 = (10, 10, 0, 0, 0, 10)^T $$

The RMP becomes:

$$ \max \lambda_1 c^T x^1 + \lambda_2 c^T x^2 + \lambda_3 c^T x^3$$

$$ s.t. \lambda_1 D x^1 + \lambda_2 D x^2 + \lambda_3 D x^3\leq 8 $$

$$ \lambda_1 + \lambda_2 + \lambda_3 = 1 $$

$$ \lambda_1, \lambda_2, \lambda_3 \geq 0 $$

which simplifies to...

$$ \max 10 \lambda_1 + 20 \lambda_2 + 30 \lambda_3 $$

$$ s.t. \,\,\,\,\quad 10 \lambda_1 \quad\quad\quad + 20 \lambda_3 \leq 8 $$

$$ \lambda_1 + \lambda_2 + \lambda_3 = 1 $$

$$ \lambda_1, \lambda_2, \lambda_3 \geq 0 $$

In [16]:
lambdas = cp.Variable(3)

c_lambdas = np.array([10, 20, 30])

D = np.array([10, 0, 20])

obj = cp.Maximize(c_lambdas @ lambdas)
prob = cp.Problem(obj, [D @ lambdas <= 8,
                        sum(lambdas) == 1,
                        lambdas >= 0])
prob.solve()

print(f'Objective function value at optimum: {round(prob.value)}')
print(f'Optimal solution: {np.round(lambdas.value,1)}^T')

Objective function value at optimum: 24.0
Optimal solution: [-0.   0.6  0.4]^T


**9) All we really need is the dual optimal solution to the RMP. So formulate the dual problem to the new RMP. Solve it in Python. Write down the dual optimal solution.**

According to the optimal solution above, we drop $\lambda_1$ and use the columns corresponding to $\lambda_2$ and $\lambda_3$.

The new RMP has the basis matrix $B = \begin{bmatrix} 0 & 20 \\ 1 & 1\end{bmatrix}$, and cost vector $c_B = [20, 30]$.

The dual variables $[\hat y^T, \hat r] = c_B^T B^{-1}$. In the cell below we will calculate the dual variables.

The dual variables $[\hat y^T, \hat r] = [0.5, 20]$.

In [18]:
c_B = np.array([20, 30])
B = np.array([[0, 20],
              [1,  1]])

dual_vars = c_B.T @ np.linalg.inv(B)
print(f'Dual Variables: {dual_vars}')

Dual Variables: [ 0.5 20. ]


**10) Formulate the subproblem that maximizes the reduced cost of the restricted master problem.**

Again the pricing problem can be formulated as:

![](3.png)

In this case, $c^T - \hat y^T D = (-0.5, 2, 0, 0, 1, 0.5)$, 

$\hat r = 20$, 

$F = \begin{bmatrix} 1 & 1 & 1 & 0 & 0 & 0 \\ 
                     0 & 0 & 0 & 1 & 1 & 1 \\
                     1 & 0 & 0 & 1 & 0 & 0 \\
                     0 & 1 & 0 & 0 & 1 & 0 \\
                     0 & 0 & 1 & 0 & 0 & 1 \\
                     \end{bmatrix}$, and

$\vec b = (20, 10, 10, 10, 10)^T$

**11) Solve the above reduced cost maximization problem in Python. Write down its optimal objective value. Should we terminate the Dantzig-Wolfe algorithm? Why?**


In [19]:
x = cp.Variable(6)

c = np.array([-0.5,2,0,0,1,0.5])

F = np.array([[1,1,1,0,0,0],
              [0,0,0,1,1,1],
              [1,0,0,1,0,0],
              [0,1,0,0,1,0],
              [0,0,1,0,0,1]])

b = np.array([20,10,10,10,10])

obj = cp.Maximize(c@x - 20)
prob = cp.Problem(obj, [F @ x == b,
                        x >= 0])
prob.solve()

print(f'Objective function value at optimum: {prob.value}')
print(f'Optimal solution: {np.round(x.value,1)}^T')

if prob.value > 0:
    print('\nAdding this solution to the RMP will increase the objective value, so we continue iterating.')
else:
    print('\nAdding this solution to the RMP will not increase the objective value, \
    so we terminate the algorithm at this iteration!')

Objective function value at optimum: -0.0
Optimal solution: [ 5. 10.  5.  5.  0.  5.]^T

Adding this solution to the RMP will not increase the objective value,     so we terminate the algorithm at this iteration!
